### Code Hist.

 - CODE  
    &ensp; : Crawling - 특일 정보 조회 (KASI)

  - DATE  
    &ensp; 2023-11-29 Created  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3)   
    
 - DESC  
    &ensp; : 전처리 - 한국지역난방공사 열판매량/열공급량   
    &emsp; 1) 결측치가 없어서, 그대로 사용  
    &emsp;&ensp;&ensp; 
    &emsp;&ensp;&ensp; (Crawl Code 없음)   

 - DATA  
    &emsp; <"Input">  
    1) None (Input Dataset)  
    &emsp;- Period :   
    &emsp;- Interval : 

    &emsp; <"Output">  
    1) Hourly (관측소/년도별 출력)  
    &nbsp;df_data_cal.to_csv(data_dir + 'KASI_DATE_D_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    2) Daily (관측소/년도별 출력)  
    &nbsp;df_data_cal_24.to_csv(data_dir + 'KASI_DATE_H_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['locdate', 'YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'HOUR', 'MINUTE']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    

 - Related Link  
    &ensp; : 

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [151]:
#region Basic_Import
## Basic
import os
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import math
import random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import train_test_split

# CLustering 알고리즘의 성능 평가 측도
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## 정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [152]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [153]:
## Import_Local
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC
from Src_Dev_Common import DEV_KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [154]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [155]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-03-26 22:18:58.992769
2024 / 3 / 26
22 : 18


In [156]:
## Dict_Domain
dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 99:"GAS"} ## GAS는 사용하지 않음.
int_domain = 0
str_domain = str(dict_domain[int_domain])

dict_col_accu = {0 : "ACTUAL_ACCU_EFF" ## ELEC
                 , 1 : "ACCU_HEAT" ## HEAT
                 , 2 : "ACCU_FLOW" ## WATER
                 , 3 : "ACCU_HEAT" ## HOT 열량
                 , 4 : "ACCU_FLOW" ## HOT 유량
                 , 99 : "ACCU_FLOW" ## GAS
                 }
str_col_accu = str(str_domain + "_" + str(dict_col_accu[int_domain]))

dict_col_inst = {0 : "INST_EFF" ## ELEC_ACCU/INST_EFF
                , 1 : "INST_HEAT" ## HEAT_ACCU/INST_HEAT
                , 2 : "INST_FLOW" ## WATER_ACCU/INST_FLOW
                , 3 : "INST_HEAT" ## HOT_ACCU/INST_HEAT
                , 4 : "INST_FLOW" ## HOT_ACCU/INST_FLOW
                , 99 : "INST_FLOW" ## GAS_ACCU/INST_FLOW
                } 
str_col_inst = str(str_domain + "_" + str(dict_col_inst[int_domain]))

## Directory Root
str_dirData = "../data/data_Energy_KIER/"
str_dir_raw = '../data/data_Energy_KIER/KIER_0_Raw/'
str_dirName_bld = '../data/data_Energy_KIER/KIER_1_BLD/'
str_dirName_f = '../data/data_Energy_KIER/KIER_2_F_' + str_domain + '/'
str_dirName_h = '../data/data_Energy_KIER/KIER_3_H_' + str_domain + '/'

## File
str_fileRaw = str('KIER_RAW_' + str_domain + '_2023-11-12.csv')
str_fileRaw_hList = str('KIER_hList_' + str_domain + '.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'Data_Population_SJ_2014_2018_1st_Interpolated.csv', 'Data_Population_SJ_2014_2018_D_1st_Interpolated.csv', 'Data_Population_SJ_2014_2018_H_1st_Interpolated.csv', 'DATE_1M_2023-10-20.csv', 'ENV_SENSOR_HST_202309251452.csv', 'F1_MAIN_POWER_HST_202309251503.csv', 'F1_SUB_POWER_HST_202309251510.csv', 'HEATPUMP_HST_202309251520.csv', 'KIER_0_Raw', 'KIER_1_BLD', 'KIER_2_F_ELEC', 'KIER_2_F_GAS', 'KIER_2_F_HEAT', 'KIER_2_F_HOT', 'KIER_2_F_WATER', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', 'LIGHT_HST_202309251523.csv', 'MACHINE_ROOM_HEAT_HST_202309251534.csv', 'MACHINE_ROOM_POWER_APT_HST_20221205_202309251617.csv', 'MACHINE_ROOM_POWER_APT_HS

In [157]:
## 10분 단위
# str_file = 'KIER_' + str_domain + '_INST_03-01_10min.csv'
## 1시간 단위
str_file = 'KIER_' + str_domain + '_INST_03-02_1H.csv'
## 1일 단위
# str_file = 'KIER_' + str_domain + '_INST_03-03_1D.csv' 
## 1주 단위
# str_file = 'KIER_' + str_domain + '_INST_03-04_1W.csv' 
## 1개월 단위
# str_file = 'KIER_' + str_domain + '_INST_03-05_1M.csv' 

str_interval = str_file[len('KIER_' + str_domain + '_INST_03-00_'):-len('.csv')]
str_file_clustering = 'KIER_' + str(str_domain) + '_Labeled_' + str_interval + '.csv'

## 01-02. Data Load (df_raw)

### 01-02-01. KDHC Heat Usage (Intergrated)

In [158]:
# str_file = 'KIER_' + str_domain + '_INST_' + str_interval + '.csv'
df_kier_raw = pd.read_csv(str_dirName_h + str_file
                          , index_col = 0)
df_kier_raw['METER_DATE'] = pd.to_datetime(df_kier_raw['METER_DATE'])
## [임시] 모든 값이 0인 마지막행 제거
df_kier_raw = df_kier_raw[:-1]

## 호실별 사용량 데이터 컬럼 리스트
list_h_usage = df_kier_raw.columns[1:]
## YMDH 컬럼 생성
df_kier_raw = com_date.create_col_ymdhm(df_kier_raw, 'METER_DATE')
## YMDH / 호실별 사용량 컬럼만 남기기
df_kier_temp = df_kier_raw[list_h_usage] 
df_kier_temp['YEAR'] = df_kier_raw['YEAR']
df_kier_temp['MONTH'] = df_kier_raw['MONTH']
df_kier_temp['DAY'] = df_kier_raw['DAY']
df_kier_temp['HOUR'] = df_kier_raw['HOUR']
df_kier_temp

,ELEC_INST_EFF_561-1-1,ELEC_INST_EFF_561-1-2,ELEC_INST_EFF_561-1-3,ELEC_INST_EFF_561-1-4,ELEC_INST_EFF_561-2-1,ELEC_INST_EFF_561-2-2,ELEC_INST_EFF_561-2-3,ELEC_INST_EFF_561-2-4,ELEC_INST_EFF_561-3-1,ELEC_INST_EFF_561-3-2,...,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,YEAR,MONTH,DAY,HOUR
0,0.8600000000,1.2500000000,0.1700000000,1.0645217391,0.8600000000,0.2500000000,0.6000000000,0.4800000000,0.8100000000,0.3200000000,...,0.2200000000,0.3600000000,0.2600000000,0.2100000000,0.1600000000,0.4400000000,2022,7,17,23
1,0.7280647787,1.1200000000,0.1800000000,0.9600000000,0.4500000000,0.2600000000,0.6000000000,0.4600000000,0.7100000000,0.4000000000,...,0.3100000000,0.4100000000,0.2500000000,0.2000000000,0.1400000000,0.3860498534,2022,7,18,0
2,0.5020212513,0.9900000000,0.1900000000,0.8900000000,0.5800000000,0.2400000000,0.6300000000,0.4500000000,0.7000000000,0.2900000000,...,0.2500000000,0.3400000000,0.2600000000,0.2000000000,0.1300000000,0.2500000000,2022,7,18,1
3,0.3300000000,1.0300000000,0.2000000000,1.0600000000,0.2700000000,0.1800000000,0.1400000000,0.2700000000,0.6100000000,0.4000000000,...,0.2100000000,0.3400000000,0.2600000000,0.1900000000,0.1400000000,0.2200000000,2022,7,18,2
4,0.2600000000,1.0000000000,0.2700000000,0.8700000000,0.6200000000,0.1700000000,0.1300000000,0.2300000000,0.5700000000,0.2800000000,...,0.1900000000,0.4600000000,0.2800000000,0.1800000000,0.1800000000,0.3000000000,2022,7,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14159,0.6700000000,0.5400000000,0.2100000000,0.4500000000,1.4700000000,0.1300000000,0.1500000000,0.4600000000,0.2900000000,0.2400000000,...,0.1400000000,0.7800000000,0.2200000000,0.2500000000,0.2800000000,0.2200000000,2024,2,27,22
14160,0.4900000000,0.3200000000,0.2200000000,0.2800000000,0.6100000000,0.1300000000,0.1000000000,0.4900000000,0.2300000000,0.2000000000,...,0.1400000000,0.6800000000,0.2600000000,0.2000000000,0.1700000000,0.2100000000,2024,2,27,23
14161,0.4600000000,0.2500000000,0.2900000000,0.1700000000,0.3900000000,0.1900000000,0.1000000000,0.3000000000,0.1900000000,0.2400000000,...,0.1500000000,0.5900000000,0.2300000000,0.1900000000,0.1500000000,0.2100000000,2024,2,28,0
14162,0.5100000000,0.2100000000,0.1900000000,0.2300000000,0.3300000000,0.1500000000,0.0900000000,0.1900000000,0.1700000000,0.2500000000,...,0.1500000000,0.6500000000,0.2300000000,0.2000000000,0.1600000000,0.1900000000,2024,2,28,1


In [159]:
df_kier_raw

,METER_DATE,ELEC_INST_EFF_561-1-1,ELEC_INST_EFF_561-1-2,ELEC_INST_EFF_561-1-3,ELEC_INST_EFF_561-1-4,ELEC_INST_EFF_561-2-1,ELEC_INST_EFF_561-2-2,ELEC_INST_EFF_561-2-3,ELEC_INST_EFF_561-2-4,ELEC_INST_EFF_561-3-1,...,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,YEAR,MONTH,DAY,HOUR,MINUTE
0,2022-07-17 23:00:00,0.8600000000,1.2500000000,0.1700000000,1.0645217391,0.8600000000,0.2500000000,0.6000000000,0.4800000000,0.8100000000,...,0.3600000000,0.2600000000,0.2100000000,0.1600000000,0.4400000000,2022,7,17,23,0
1,2022-07-18 00:00:00,0.7280647787,1.1200000000,0.1800000000,0.9600000000,0.4500000000,0.2600000000,0.6000000000,0.4600000000,0.7100000000,...,0.4100000000,0.2500000000,0.2000000000,0.1400000000,0.3860498534,2022,7,18,0,0
2,2022-07-18 01:00:00,0.5020212513,0.9900000000,0.1900000000,0.8900000000,0.5800000000,0.2400000000,0.6300000000,0.4500000000,0.7000000000,...,0.3400000000,0.2600000000,0.2000000000,0.1300000000,0.2500000000,2022,7,18,1,0
3,2022-07-18 02:00:00,0.3300000000,1.0300000000,0.2000000000,1.0600000000,0.2700000000,0.1800000000,0.1400000000,0.2700000000,0.6100000000,...,0.3400000000,0.2600000000,0.1900000000,0.1400000000,0.2200000000,2022,7,18,2,0
4,2022-07-18 03:00:00,0.2600000000,1.0000000000,0.2700000000,0.8700000000,0.6200000000,0.1700000000,0.1300000000,0.2300000000,0.5700000000,...,0.4600000000,0.2800000000,0.1800000000,0.1800000000,0.3000000000,2022,7,18,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14159,2024-02-27 22:00:00,0.6700000000,0.5400000000,0.2100000000,0.4500000000,1.4700000000,0.1300000000,0.1500000000,0.4600000000,0.2900000000,...,0.7800000000,0.2200000000,0.2500000000,0.2800000000,0.2200000000,2024,2,27,22,0
14160,2024-02-27 23:00:00,0.4900000000,0.3200000000,0.2200000000,0.2800000000,0.6100000000,0.1300000000,0.1000000000,0.4900000000,0.2300000000,...,0.6800000000,0.2600000000,0.2000000000,0.1700000000,0.2100000000,2024,2,27,23,0
14161,2024-02-28 00:00:00,0.4600000000,0.2500000000,0.2900000000,0.1700000000,0.3900000000,0.1900000000,0.1000000000,0.3000000000,0.1900000000,...,0.5900000000,0.2300000000,0.1900000000,0.1500000000,0.2100000000,2024,2,28,0,0
14162,2024-02-28 01:00:00,0.5100000000,0.2100000000,0.1900000000,0.2300000000,0.3300000000,0.1500000000,0.0900000000,0.1900000000,0.1700000000,...,0.6500000000,0.2300000000,0.2000000000,0.1600000000,0.1900000000,2024,2,28,1,0


### 01-02-02. KMA ASOS

In [160]:
str_dir_kmaAsos = "../data/data_KMA_ASOS/"
print(os.listdir(str_dir_kmaAsos))

Data_ASOS = pd.read_csv(str_dir_kmaAsos + 'ASOS_119_2010-2024_HR.csv', index_col = 0)

['108_SEOUL', '119_SUWON', '131_CHEONGJU', '143_DAEGU', '156_GWANGJU_JEONNAM', '253_KIMHAE', '257_YANGSAN', '99_PAJU', 'ASOS_108_2010-2023_HR.csv', 'ASOS_119_2010-2024_HR.csv', 'ASOS_119_2010-2024_HR_INTP.csv', 'ASOS_119_2010_HR.csv', 'ASOS_119_2011_HR.csv', 'ASOS_119_2012_HR.csv', 'ASOS_119_2013_HR.csv', 'ASOS_119_2014_HR.csv', 'ASOS_119_2015_HR.csv', 'ASOS_119_2016_HR.csv', 'ASOS_119_2017_HR.csv', 'ASOS_119_2018_HR.csv', 'ASOS_119_2019_HR.csv', 'ASOS_119_2020_HR.csv', 'ASOS_119_2021_HR.csv', 'ASOS_119_2022_HR.csv', 'ASOS_119_2023_HR.csv', 'ASOS_119_2024-2024_HR.csv', 'ASOS_119_2024_HR.csv', 'ASOS_119_2024_HR_DOWN.csv', 'ASOS_131_2010-2023_HR.csv', 'ASOS_143_2010-2023_HR.csv', 'ASOS_156_2010-2023_HR.csv', 'ASOS_253_2010-2023_HR.csv', 'ASOS_2940_2010_HR.csv', 'ASOS_99_2010-2023_HR.csv']


In [161]:
## Date 형식 지정
# Data_ASOS['METER_DATE'] = pd.to_datetime(Data_ASOS[str_domain + '_METER_DATE'])

## Data 보간
Data_ASOS['temp_outdoor'] = Data_ASOS['temp_outdoor'].interpolate()
Data_ASOS['temp_dew_point'] = Data_ASOS['temp_dew_point'].interpolate()
Data_ASOS['temp_ground'] = Data_ASOS['temp_ground'].interpolate()
Data_ASOS['humidity'] = Data_ASOS['humidity'].interpolate()
Data_ASOS['wind_speed'] = Data_ASOS['wind_speed'].interpolate()
Data_ASOS['wind_direction'] = Data_ASOS['wind_direction'].interpolate()
Data_ASOS['pressure_vapor'] = Data_ASOS['pressure_vapor'].interpolate()
Data_ASOS['pressure_area'] = Data_ASOS['pressure_area'].interpolate()
Data_ASOS['pressure_sea'] = Data_ASOS['pressure_sea'].interpolate()

## Data 0으로 채움
Data_ASOS['rainfall'] = Data_ASOS['rainfall'].fillna(0)
Data_ASOS['snowfall'] = Data_ASOS['snowfall'].fillna(0)
Data_ASOS['snowfall_3hr'] = Data_ASOS['snowfall_3hr'].fillna(0)
Data_ASOS['sunshine'] = Data_ASOS['sunshine'].fillna(0)
Data_ASOS['solar_radiation'] = Data_ASOS['solar_radiation'].fillna(0)
Data_ASOS['cloud_total'] = Data_ASOS['cloud_total'].fillna(0)
Data_ASOS['cloud_midlow'] = Data_ASOS['cloud_midlow'].fillna(0)
Data_ASOS['visual_range'] = Data_ASOS['visual_range'].fillna(0)

print(Data_ASOS.info())
Data_ASOS

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124464 entries, 0 to 124463
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   METER_DATE            124464 non-null  object 
 1   day_of_the_week       124464 non-null  object 
 2   YEAR                  124464 non-null  int64  
 3   MONTH                 124464 non-null  int64  
 4   DAY                   124464 non-null  int64  
 5   HOUR                  124464 non-null  int64  
 6   MINUTE                124464 non-null  int64  
 7   code_day_of_the_week  124464 non-null  int64  
 8   temp_outdoor          124464 non-null  float64
 9   temp_dew_point        124464 non-null  float64
 10  temp_ground           124464 non-null  float64
 11  humidity              124464 non-null  float64
 12  rainfall              124464 non-null  float64
 13  snowfall              124464 non-null  float64
 14  snowfall_3hr          124464 non-null  float64
 15  

,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,temp_outdoor,temp_dew_point,...,wind_speed,wind_direction,pressure_vapor,pressure_area,pressure_sea,sunshine,solar_radiation,cloud_total,cloud_midlow,visual_range
0,2010-01-01 00:00:00,Friday,2010,1,1,0,0,4,-10.3000000000,-17.6000000000,...,1.6000000000,340.0000000000,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000
1,2010-01-01 01:00:00,Friday,2010,1,1,1,0,4,-10.4000000000,-17.9000000000,...,0.3000000000,0.0000000000,1.5000000000,1019.5000000000,1024.1000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,2010-01-01 02:00:00,Friday,2010,1,1,2,0,4,-10.6000000000,-17.6000000000,...,0.6000000000,270.0000000000,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,2010-01-01 03:00:00,Friday,2010,1,1,3,0,4,-11.0000000000,-18.0000000000,...,1.5000000000,290.0000000000,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000
4,2010-01-01 04:00:00,Friday,2010,1,1,4,0,4,-11.6000000000,-18.2000000000,...,0.9000000000,290.0000000000,1.5000000000,1019.4000000000,1024.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124459,2024-03-13 19:00:00,Wednesday,2024,3,13,19,0,2,6.8000000000,-1.6000000000,...,2.3000000000,270.0000000000,5.4000000000,1016.2000000000,1021.1000000000,0.1000000000,0.0600000000,0.0000000000,0.0000000000,2085.0000000000
124460,2024-03-13 20:00:00,Wednesday,2024,3,13,20,0,2,6.2000000000,-2.6000000000,...,1.0000000000,270.0000000000,5.0000000000,1016.3000000000,1021.2000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2180.0000000000
124461,2024-03-13 21:00:00,Wednesday,2024,3,13,21,0,2,5.2000000000,-2.1000000000,...,0.7000000000,200.0000000000,5.2000000000,1016.7000000000,1021.7000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1978.0000000000
124462,2024-03-13 22:00:00,Wednesday,2024,3,13,22,0,2,4.6000000000,-1.0000000000,...,1.1000000000,160.0000000000,5.7000000000,1016.8000000000,1021.8000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1670.0000000000


In [162]:
## 날씨 데이터 컬럼 리스트
list_weather = Data_ASOS.columns[1:]
## YMDH 컬럼 생성
Data_ASOS = com_date.create_col_ymdhm(Data_ASOS, 'METER_DATE')
## YMDH / 호실별 사용량 컬럼만 남기기
Data_ASOS_temp = Data_ASOS[list_weather] 
Data_ASOS_temp['YEAR'] = Data_ASOS['YEAR']
Data_ASOS_temp['MONTH'] = Data_ASOS['MONTH']
Data_ASOS_temp['DAY'] = Data_ASOS['DAY']
Data_ASOS_temp['HOUR'] = Data_ASOS['HOUR']

print(Data_ASOS_temp.info())
Data_ASOS_temp

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124464 entries, 0 to 124463
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   day_of_the_week       124464 non-null  object 
 1   YEAR                  124464 non-null  int64  
 2   MONTH                 124464 non-null  int64  
 3   DAY                   124464 non-null  int64  
 4   HOUR                  124464 non-null  int64  
 5   MINUTE                124464 non-null  int64  
 6   code_day_of_the_week  124464 non-null  int64  
 7   temp_outdoor          124464 non-null  float64
 8   temp_dew_point        124464 non-null  float64
 9   temp_ground           124464 non-null  float64
 10  humidity              124464 non-null  float64
 11  rainfall              124464 non-null  float64
 12  snowfall              124464 non-null  float64
 13  snowfall_3hr          124464 non-null  float64
 14  wind_speed            124464 non-null  float64
 15  

,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,temp_outdoor,temp_dew_point,temp_ground,...,wind_speed,wind_direction,pressure_vapor,pressure_area,pressure_sea,sunshine,solar_radiation,cloud_total,cloud_midlow,visual_range
0,Friday,2010,1,1,0,0,4,-10.3000000000,-17.6000000000,-5.8000000000,...,1.6000000000,340.0000000000,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000
1,Friday,2010,1,1,1,0,4,-10.4000000000,-17.9000000000,-5.7000000000,...,0.3000000000,0.0000000000,1.5000000000,1019.5000000000,1024.1000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,Friday,2010,1,1,2,0,4,-10.6000000000,-17.6000000000,-5.9000000000,...,0.6000000000,270.0000000000,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,Friday,2010,1,1,3,0,4,-11.0000000000,-18.0000000000,-5.9000000000,...,1.5000000000,290.0000000000,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000
4,Friday,2010,1,1,4,0,4,-11.6000000000,-18.2000000000,-6.0000000000,...,0.9000000000,290.0000000000,1.5000000000,1019.4000000000,1024.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124459,Wednesday,2024,3,13,19,0,2,6.8000000000,-1.6000000000,4.9000000000,...,2.3000000000,270.0000000000,5.4000000000,1016.2000000000,1021.1000000000,0.1000000000,0.0600000000,0.0000000000,0.0000000000,2085.0000000000
124460,Wednesday,2024,3,13,20,0,2,6.2000000000,-2.6000000000,3.3000000000,...,1.0000000000,270.0000000000,5.0000000000,1016.3000000000,1021.2000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2180.0000000000
124461,Wednesday,2024,3,13,21,0,2,5.2000000000,-2.1000000000,2.0000000000,...,0.7000000000,200.0000000000,5.2000000000,1016.7000000000,1021.7000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1978.0000000000
124462,Wednesday,2024,3,13,22,0,2,4.6000000000,-1.0000000000,1.4000000000,...,1.1000000000,160.0000000000,5.7000000000,1016.8000000000,1021.8000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1670.0000000000


In [150]:
Data_ASOS_temp = com_date.create_col_datetime(Data_ASOS_temp, 'METER_DATE'
                                              , 'YEAR', 'MONTH', 'DAY'
                                              , 'HOUR')
Data_ASOS_temp.to_csv(str_dir_kmaAsos + 'ASOS_119_2010-2024_HR_INTP.csv')
# Data_ASOS.to_csv(str_dir_kmaAsos + 'ASOS_119_2010-2024_HR_INTP.csv')
# Data_ASOS
Data_ASOS_temp

,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,temp_outdoor,temp_dew_point,temp_ground,...,pressure_vapor,pressure_area,pressure_sea,sunshine,solar_radiation,cloud_total,cloud_midlow,visual_range,None,METER_DATE
0,Friday,2010,1,1,0,0,4,-10.3000000000,-17.6000000000,-5.8000000000,...,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000,0,2010-01-01 00:00:00
1,Friday,2010,1,1,1,0,4,-10.4000000000,-17.9000000000,-5.7000000000,...,1.5000000000,1019.5000000000,1024.1000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,2010-01-01 01:00:00
2,Friday,2010,1,1,2,0,4,-10.6000000000,-17.6000000000,-5.9000000000,...,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0,2010-01-01 02:00:00
3,Friday,2010,1,1,3,0,4,-11.0000000000,-18.0000000000,-5.9000000000,...,1.5000000000,1019.7000000000,1024.3000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000,0,2010-01-01 03:00:00
4,Friday,2010,1,1,4,0,4,-11.6000000000,-18.2000000000,-6.0000000000,...,1.5000000000,1019.4000000000,1024.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2000.0000000000,0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124459,Wednesday,2024,3,13,19,0,2,6.8000000000,-1.6000000000,4.9000000000,...,5.4000000000,1016.2000000000,1021.1000000000,0.1000000000,0.0600000000,0.0000000000,0.0000000000,2085.0000000000,0,2024-03-13 19:00:00
124460,Wednesday,2024,3,13,20,0,2,6.2000000000,-2.6000000000,3.3000000000,...,5.0000000000,1016.3000000000,1021.2000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,2180.0000000000,0,2024-03-13 20:00:00
124461,Wednesday,2024,3,13,21,0,2,5.2000000000,-2.1000000000,2.0000000000,...,5.2000000000,1016.7000000000,1021.7000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1978.0000000000,0,2024-03-13 21:00:00
124462,Wednesday,2024,3,13,22,0,2,4.6000000000,-1.0000000000,1.4000000000,...,5.7000000000,1016.8000000000,1021.8000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1670.0000000000,0,2024-03-13 22:00:00


In [163]:
Data_ASOS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124464 entries, 0 to 124463
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   METER_DATE            124464 non-null  datetime64[ns]
 1   day_of_the_week       124464 non-null  object        
 2   YEAR                  124464 non-null  int64         
 3   MONTH                 124464 non-null  int64         
 4   DAY                   124464 non-null  int64         
 5   HOUR                  124464 non-null  int64         
 6   MINUTE                124464 non-null  int64         
 7   code_day_of_the_week  124464 non-null  int64         
 8   temp_outdoor          124464 non-null  float64       
 9   temp_dew_point        124464 non-null  float64       
 10  temp_ground           124464 non-null  float64       
 11  humidity              124464 non-null  float64       
 12  rainfall              124464 non-null  float64       
 13 

## 01-03. Data Integeration

In [164]:
df_merged = pd.merge(df_kier_temp, Data_ASOS_temp
                     , how = 'left', on = ['YEAR', 'MONTH', 'DAY', 'HOUR'])
df_merged = com_date.create_col_datetime(df_tar = df_merged, col_tar = 'METER_DATE'
                                         , col_y = 'YEAR', col_m = 'MONTH', col_d = 'DAY'
                                         , col_h = 'HOUR')

## 컬럼 재배치
list_date = ['METER_DATE']
df_merged = df_merged[list_date + list_weather.to_list() + list_h_usage.to_list()]
df_merged

,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,temp_outdoor,temp_dew_point,...,ELEC_INST_EFF_563-22-3,ELEC_INST_EFF_563-22-4,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2
0,2022-07-17 23:00:00,Sunday,2022,7,17,23,0,6,24.5000000000,21.0000000000,...,0.6000000000,0.6100000000,0.1900000000,0.2200000000,0.2200000000,0.3600000000,0.2600000000,0.2100000000,0.1600000000,0.4400000000
1,2022-07-18 00:00:00,Monday,2022,7,18,0,0,0,24.0000000000,21.3000000000,...,0.2373076923,0.4600000000,0.1600000000,0.2400000000,0.3100000000,0.4100000000,0.2500000000,0.2000000000,0.1400000000,0.3860498534
2,2022-07-18 01:00:00,Monday,2022,7,18,1,0,0,23.7000000000,21.7000000000,...,0.2100000000,0.2700000000,0.1600000000,0.2700000000,0.2500000000,0.3400000000,0.2600000000,0.2000000000,0.1300000000,0.2500000000
3,2022-07-18 02:00:00,Monday,2022,7,18,2,0,0,24.0000000000,22.2000000000,...,0.2000000000,0.2500000000,0.1400000000,0.2400000000,0.2100000000,0.3400000000,0.2600000000,0.1900000000,0.1400000000,0.2200000000
4,2022-07-18 03:00:00,Monday,2022,7,18,3,0,0,23.9000000000,22.3000000000,...,0.2100000000,0.2800000000,0.1300000000,0.3200000000,0.1900000000,0.4600000000,0.2800000000,0.1800000000,0.1800000000,0.3000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14159,2024-02-27 22:00:00,Tuesday,2024,2,27,22,0,1,6.1000000000,-3.5000000000,...,0.0800000000,0.3900000000,0.1800000000,0.1800000000,0.1400000000,0.7800000000,0.2200000000,0.2500000000,0.2800000000,0.2200000000
14160,2024-02-27 23:00:00,Tuesday,2024,2,27,23,0,1,5.9000000000,-3.7000000000,...,0.1100000000,0.3200000000,0.1800000000,0.1600000000,0.1400000000,0.6800000000,0.2600000000,0.2000000000,0.1700000000,0.2100000000
14161,2024-02-28 00:00:00,Wednesday,2024,2,28,0,0,2,5.2000000000,-3.8000000000,...,0.1000000000,0.3300000000,0.1800000000,0.1700000000,0.1500000000,0.5900000000,0.2300000000,0.1900000000,0.1500000000,0.2100000000
14162,2024-02-28 01:00:00,Wednesday,2024,2,28,1,0,2,4.5000000000,-3.5000000000,...,0.0700000000,0.3100000000,0.1800000000,0.1500000000,0.1500000000,0.6500000000,0.2300000000,0.2000000000,0.1600000000,0.1900000000


In [165]:
str_file = 'KIER_' + str_domain + '_INST_Weather_ALL_' + str_interval + '.csv'
df_merged.to_csv(str_dirName_h + str_file)

In [166]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14164 entries, 0 to 14163
Columns: 373 entries, METER_DATE to ELEC_INST_EFF_563-24-2
dtypes: float64(365), int64(6), object(2)
memory usage: 40.4+ MB


In [167]:
df_merged.isna().sum()

METER_DATE                0
day_of_the_week           0
YEAR                      0
MONTH                     0
DAY                       0
                         ..
ELEC_INST_EFF_563-23-2    0
ELEC_INST_EFF_563-23-3    0
ELEC_INST_EFF_563-23-4    0
ELEC_INST_EFF_563-24-1    0
ELEC_INST_EFF_563-24-2    0
Length: 373, dtype: int64

In [168]:
df_merged

,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,temp_outdoor,temp_dew_point,...,ELEC_INST_EFF_563-22-3,ELEC_INST_EFF_563-22-4,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2
0,2022-07-17 23:00:00,Sunday,2022,7,17,23,0,6,24.5000000000,21.0000000000,...,0.6000000000,0.6100000000,0.1900000000,0.2200000000,0.2200000000,0.3600000000,0.2600000000,0.2100000000,0.1600000000,0.4400000000
1,2022-07-18 00:00:00,Monday,2022,7,18,0,0,0,24.0000000000,21.3000000000,...,0.2373076923,0.4600000000,0.1600000000,0.2400000000,0.3100000000,0.4100000000,0.2500000000,0.2000000000,0.1400000000,0.3860498534
2,2022-07-18 01:00:00,Monday,2022,7,18,1,0,0,23.7000000000,21.7000000000,...,0.2100000000,0.2700000000,0.1600000000,0.2700000000,0.2500000000,0.3400000000,0.2600000000,0.2000000000,0.1300000000,0.2500000000
3,2022-07-18 02:00:00,Monday,2022,7,18,2,0,0,24.0000000000,22.2000000000,...,0.2000000000,0.2500000000,0.1400000000,0.2400000000,0.2100000000,0.3400000000,0.2600000000,0.1900000000,0.1400000000,0.2200000000
4,2022-07-18 03:00:00,Monday,2022,7,18,3,0,0,23.9000000000,22.3000000000,...,0.2100000000,0.2800000000,0.1300000000,0.3200000000,0.1900000000,0.4600000000,0.2800000000,0.1800000000,0.1800000000,0.3000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14159,2024-02-27 22:00:00,Tuesday,2024,2,27,22,0,1,6.1000000000,-3.5000000000,...,0.0800000000,0.3900000000,0.1800000000,0.1800000000,0.1400000000,0.7800000000,0.2200000000,0.2500000000,0.2800000000,0.2200000000
14160,2024-02-27 23:00:00,Tuesday,2024,2,27,23,0,1,5.9000000000,-3.7000000000,...,0.1100000000,0.3200000000,0.1800000000,0.1600000000,0.1400000000,0.6800000000,0.2600000000,0.2000000000,0.1700000000,0.2100000000
14161,2024-02-28 00:00:00,Wednesday,2024,2,28,0,0,2,5.2000000000,-3.8000000000,...,0.1000000000,0.3300000000,0.1800000000,0.1700000000,0.1500000000,0.5900000000,0.2300000000,0.1900000000,0.1500000000,0.2100000000
14162,2024-02-28 01:00:00,Wednesday,2024,2,28,1,0,2,4.5000000000,-3.5000000000,...,0.0700000000,0.3100000000,0.1800000000,0.1500000000,0.1500000000,0.6500000000,0.2300000000,0.2000000000,0.1600000000,0.1900000000
